In [1]:
import os
import copy

import bdpy
import numpy as np
import hdf5storage

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
bdata = bdpy.BData('../data/fmri/visrecon_s1.mat')

In [4]:
bdata.show_metadata()

| Key          | Description            |
|--------------|------------------------|
| VoxelData    | Index for voxel data   |
| Run          | Index for run number   |
| Block        | Index for block number |
| voxel_x      | Voxel x coordinate     |
| voxel_y      | Voxel y coordinate     |
| voxel_z      | Voxel z coordinate     |
| L_V1d        | Index for ROI L_V1d    |
| L_V1v        | Index for ROI L_V1v    |
| L_V2d        | Index for ROI L_V2d    |
| L_V2v        | Index for ROI L_V2v    |
| L_V3         | Index for ROI L_V3     |
| L_V3A        | Index for ROI L_V3A    |
| L_V4v        | Index for ROI L_V4v    |
| L_VP         | Index for ROI L_VP     |
| R_V1d        | Index for ROI R_V1d    |
| R_V1v        | Index for ROI R_V1v    |
| R_V2d        | Index for ROI R_V2d    |
| R_V2v        | Index for ROI R_V2v    |
| R_V3         | Index for ROI R_V3     |
| R_V3A        | Index for ROI R_V3A    |
| R_V4v        | Index for ROI R_V4v    |
| R_VP         | Index for ROI R_V

In [5]:
stimulus_type = bdata.select('StimulusType').flatten()
stimulus_type.shape

(1172,)

In [6]:
# Collect stimulus information

# stimulus_name: List of stimulus IDs for each trial
# stimulus_vmap: Dict mapping stimulus ID to human-readable stimulus name
# stimulus_images: Dict mapping stimulus ID to stimulus image

stimulus_name   = []
stimulus_vmap   = {}
stimulus_images = {}


def is_in_dict_array(a: np.ndarray, d: dict):
    for k, v in d.items():
        if np.array_equal(a, v):
            return True
    return False


stimulus_id = 1

for i in range(bdata.dataset.shape[0]):
    if stimulus_type[i] < 0:
        stimulus_name.append(0)
        continue

    img = bdata.select("Image")[i, :]
    img = np.reshape(img, (10, 10), order="F")

    if is_in_dict_array(img, stimulus_images):

        names = [k for k, v in stimulus_images.items() if np.array_equal(img, v)]
        if len(names) != 1:
            raise RuntimeError(f"Invalid image, hit {len(names)}")
        name = names[0]
        stimulus_name.append(name)
        continue
        
    # Random pattern
    if stimulus_type[i] == 1:
        name = f"random_s1-{i:04}"
    
    # Geometric shapes
    if stimulus_type[i] == 2:
        name = f"geometric-{i:04}"

    # Alphabets
    if stimulus_type[i] == 5:
        name = f"alphabets-{i:04}"

    # Alphabets thin
    if stimulus_type[i] == 3:
        name = f"alphabets_thin-{i:04}"

    # Alphabets long
    if stimulus_type[i] == 4:
        name = f"alphabets_long-{i:04}"

    stimulus_images.update({stimulus_id: img})
    stimulus_vmap.update({stimulus_id: name})
    stimulus_name.append(stimulus_id)
    
    stimulus_id += 1

stimulus_vmap.update({0: 'n/a'})

len(stimulus_name)   # -> 1172
len(stimulus_images) # -> 440 + 20
len(stimulus_vmap)   # -> 440 + 20 + 1

1172

460

461

In [7]:
# Rename test stimulus

# Note: this mapping may differ across subjects.
rename_map = {
    'geometric-0093': 'geometric-square',
    'geometric-0095': 'geometric-small_frame',
    'geometric-0097': 'geometric-plus',
    'geometric-0099': 'geometric-X',
    'geometric-0101': 'geometric-large_frame',
    'alphabets_long-0432': 'alphabets_long-n',
    'alphabets_long-0434': 'alphabets_long-e',
    'alphabets_long-0436': 'alphabets_long-u',
    'alphabets_long-0438': 'alphabets_long-r',
    'alphabets_long-0440': 'alphabets_long-o',
    'alphabets-0545': 'alphabets-n',
    'alphabets-0547': 'alphabets-e',
    'alphabets-0549': 'alphabets-u',
    'alphabets-0551': 'alphabets-r',
    'alphabets-0553': 'alphabets-o',
    'alphabets_thin-0319': 'alphabets_thin-n',
    'alphabets_thin-0321': 'alphabets_thin-e',
    'alphabets_thin-0323': 'alphabets_thin-u',
    'alphabets_thin-0325': 'alphabets_thin-r',
    'alphabets_thin-0327': 'alphabets_thin-o',
}

stimulus_vmap_renamed = {}
for k, v in stimulus_vmap.items():
    if v in rename_map:
        v = rename_map[v]
    stimulus_vmap_renamed.update({k: v})

stimulus_vmap = stimulus_vmap_renamed

In [8]:
# Save stimulus images

for stim_id, img in stimulus_images.items():
    stim_name = stimulus_vmap[stim_id]
    
    if "random" in stim_name:
        outdir = "random"
    else:
        outdir = "test"

    output_file = f"../data/stimulus/{outdir}/1x1/{stim_name}.mat"
    if os.path.exists(output_file):
        continue
        
    os.makedirs(f"../data/stimulus/{outdir}/1x1", exist_ok=True)

    img = img[np.newaxis, :]
    hdf5storage.write(img, 'feat', output_file, matlab_compatible=True)
    print(f"Saved {output_file}")

/home/kujaku/libraries/python/anaconda3-2020.11/lib/python3.8/site-packages/hdf5storage/__init__.py:1306: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  f = h5py.File(filename)


Saved ../data/stimulus/random/1x1/random_s1-0001.mat
Saved ../data/stimulus/random/1x1/random_s1-0003.mat
Saved ../data/stimulus/random/1x1/random_s1-0005.mat
Saved ../data/stimulus/random/1x1/random_s1-0007.mat
Saved ../data/stimulus/random/1x1/random_s1-0009.mat
Saved ../data/stimulus/random/1x1/random_s1-0011.mat
Saved ../data/stimulus/random/1x1/random_s1-0013.mat
Saved ../data/stimulus/random/1x1/random_s1-0015.mat
Saved ../data/stimulus/random/1x1/random_s1-0017.mat
Saved ../data/stimulus/random/1x1/random_s1-0019.mat
Saved ../data/stimulus/random/1x1/random_s1-0021.mat
Saved ../data/stimulus/random/1x1/random_s1-0023.mat
Saved ../data/stimulus/random/1x1/random_s1-0025.mat
Saved ../data/stimulus/random/1x1/random_s1-0027.mat
Saved ../data/stimulus/random/1x1/random_s1-0029.mat
Saved ../data/stimulus/random/1x1/random_s1-0031.mat
Saved ../data/stimulus/random/1x1/random_s1-0033.mat
Saved ../data/stimulus/random/1x1/random_s1-0035.mat
Saved ../data/stimulus/random/1x1/random_s1-00

In [9]:
# Add stimulus labels in Bdata

bdata.add(np.array(stimulus_name), 'stimulus_name')
bdata.add_vmap('stimulus_name', stimulus_vmap)

In [10]:
# Make training (random) fMRI data

bdata_random = copy.deepcopy(bdata)

bdata_random.dataset = bdata.dataset[stimulus_type == 1, :]
bdata_random.dataset.shape

bdata_random.save('../data/fmri/visrecon_s1_random.h5')

(440, 3674)

In [11]:
# Make teset fMRI data

bdata_test = copy.deepcopy(bdata)

index = np.logical_or(np.logical_or(np.logical_or(stimulus_type == 2, stimulus_type == 3), stimulus_type == 4), stimulus_type == 5)

bdata_test.dataset = bdata.dataset[index, :]
bdata_test.dataset.shape

bdata_test.save('../data/fmri/visrecon_s1_test.h5')

(120, 3674)